In [1]:
import konlpy
from konlpy.tag import Okt
import json
import os
from pprint import pprint
import nltk
import numpy as np
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras.models import load_model
import warnings
warnings.filterwarnings(action='ignore')

/Users/choejuwon/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/choejuwon/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/choejuwon/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/choejuwon/opt/anaconda3/lib/python3.7/site-packages

In [2]:
cat /Users/choejuwon/Desktop/voice_inside/nsmc-master/ratings_train.txt | head -n 10

id	document	label
9976970	아 더빙.. 진짜 짜증나네요 목소리	0
3819312	흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나	1
10265843	너무재밓었다그래서보는것을추천한다	0
9045019	교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정	0
6483659	사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다	1
5403919	막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.	0
7797314	원작의 긴장감을 제대로 살려내지못했다.	0
9443947	별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네	0
7156791	액션이 없는데도 재미 있는 몇안되는 영화	1
cat: stdout: Broken pipe


In [3]:
def read_data(filename):
    with open(filename, 'r') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        # txt 파일의 헤더(id document label)는 제외하기
        data = data[1:]
    return data

def tokenize(doc):
    # norm은 정규화, stem은 근어로 표시하기를 나타냄
    return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]

if os.path.isfile('/Users/choejuwon/Desktop/voice_inside/nsmc-master/datasets/ncms/train_docs.json'):
    with open('/Users/choejuwon/Desktop/voice_inside/nsmc-master/datasets/ncms/train_docs.json') as f:
        train_docs = json.load(f)
    with open('/Users/choejuwon/Desktop/voice_inside/nsmc-master/datasets/ncms/test_docs.json') as f:
        test_docs = json.load(f)
else:
    train_docs = [(tokenize(row[1]), row[2]) for row in train_data]
    test_docs = [(tokenize(row[1]), row[2]) for row in test_data]
    # JSON 파일로 저장
    with open('/Users/choejuwon/Desktop/voice_inside/nsmc-master/datasets/ncms/train_docs.json', 'w', encoding="utf-8") as make_file:
        json.dump(train_docs, make_file, ensure_ascii=False, indent="\t")
    with open('/Users/choejuwon/Desktop/voice_inside/nsmc-master/datasets/ncms/test_docs.json', 'w', encoding="utf-8") as make_file:
        json.dump(test_docs, make_file, ensure_ascii=False, indent="\t")
        
# 각 단어의 갯수를 넣어주면서 one-hot은 아니지만 zero가 거의 대부분인 vector를 만들어주고 있다.
def term_frequency(doc):
    return [doc.count(word) for word in selected_words]

def predict_pos_neg(review):
    token = tokenize(review)
    tf = term_frequency(token)
    data = np.expand_dims(np.asarray(tf).astype('float32'), axis=0)
    score = float(model.predict(data))
    if(score > 0.7):
        print("[{}]는 {:.2f}% 확률로 < 긍정적 > 입니다 ^^".format(review, score * 100))
    elif (score < 0.4) :
        print("[{}]는 {:.2f}% 확률로 < 부정적 > 입니다 ㅠㅠ".format(review, (1 - score) * 100))
    else:
        print("[{}]는 {:.2f}% 확률로 < 중립 > 입니다.".format(review, (1 - score) * 100))
        
okt = Okt()

In [4]:
train_data = read_data('/Users/choejuwon/Desktop/voice_inside/nsmc-master/ratings_train.txt')
test_data = read_data('/Users/choejuwon/Desktop/voice_inside/nsmc-master/ratings_test.txt')

In [5]:
print(len(train_data))
print(len(train_data[0]))
print(len(test_data))
print(len(test_data[0]))

150000
3
50000
3


In [6]:
train_data[:10]

[['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0'],
 ['3819312', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1'],
 ['10265843', '너무재밓었다그래서보는것을추천한다', '0'],
 ['9045019', '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '0'],
 ['6483659',
  '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다',
  '1'],
 ['5403919', '막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.', '0'],
 ['7797314', '원작의 긴장감을 제대로 살려내지못했다.', '0'],
 ['9443947',
  '별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네',
  '0'],
 ['7156791', '액션이 없는데도 재미 있는 몇안되는 영화', '1'],
 ['5912145', '왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?', '1']]

In [7]:
# 모든 sentence의 token들을 모아 놓은 것
tokens = [t for d in train_docs for t in d[0]]
print(len(tokens))

2159921


In [8]:
text = nltk.Text(tokens, name='NMSC')
print(text)

<Text: NMSC>


In [9]:
# 전체 토큰의 개수
print(len(text.tokens))

# 중복을 제외한 토큰의 개수
print(len(set(text.tokens)))            

# 출현 빈도가 높은 상위 토큰 10개
pprint(text.vocab().most_common(10))

2159921
49895
[('./Punctuation', 67778),
 ('영화/Noun', 50818),
 ('하다/Verb', 41209),
 ('이/Josa', 38540),
 ('보다/Verb', 38538),
 ('의/Josa', 30188),
 ('../Punctuation', 29055),
 ('가/Josa', 26627),
 ('에/Josa', 26468),
 ('을/Josa', 23118)]


In [10]:
# 시간 오래 걸림 주의!
# 가장 많이 나오는 상위 2000개의 데이터 선택
selected_words = [f[0] for f in text.vocab().most_common(2000)]
##### ==>>>>>> selected_words list를 저장해놓고 불러오면 될 듯!

# docs에 [[copus], label] 이런식으로 들어가 있음
train_x = [term_frequency(d) for d, _ in train_docs]
test_x = [term_frequency(d) for d, _ in test_docs]
train_y = [c for _, c in train_docs]
test_y = [c for _, c in test_docs]

In [11]:
x_train = np.asarray(train_x).astype('float32')
x_test = np.asarray(test_x).astype('float32')

y_train = np.asarray(train_y).astype('float32')
y_test = np.asarray(test_y).astype('float32')

In [12]:
# 모델 만들기
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(2000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
             loss=losses.binary_crossentropy,
             metrics=[metrics.binary_accuracy])

model.fit(x_train, y_train, epochs=10, batch_size=512)
results = model.evaluate(x_test, y_test)

# 만들어진 모델 저장
model.save('/Users/choejuwon/Desktop/voice_inside/model/model_2000.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/10
150000/150000 [==============================] - 6s 38us/sample - loss: 0.4112 - binary_accuracy: 0.8186
Epoch 2/10
150000/150000 [==============================] - 4s 26us/sample - loss: 0.3545 - binary_accuracy: 0.8408
Epoch 3/10
150000/150000 [==============================] - 4s 25us/sample - loss: 0.3362 - binary_accuracy: 0.8510
Epoch 4/10
150000/150000 [==============================] - 4s 25us/sample - loss: 0.3215 - binary_accuracy: 0.8603
Epoch 5/10
150000/150000 [==============================] - 4s 25us/sample - loss: 0.3078 - binary_accuracy: 0.8687
Epoch 6/10
150000/150000 [==============================] - 4s 25us/sample - loss: 0.2944 - binary_accuracy: 0.8756
Epoch 7/10
150000/150000 [==============================] - 4s 26us/sample - loss: 0.2810 - 

In [13]:
predict_pos_neg("올해 최고의 영화! 세 번 넘게 봐도 질리지가 않네요.")
predict_pos_neg("배경 음악이 영화의 분위기랑 너무 안 맞았습니다. 몰입에 방해가 됩니다.")
predict_pos_neg("주연 배우가 신인인데 연기를 진짜 잘 하네요. 몰입감 ㅎㄷㄷ")
predict_pos_neg("믿고 보는 감독이지만 이번에는 아니네요")
predict_pos_neg("주연배우 때문에 봤어요")
predict_pos_neg("오늘 뭐먹지?")

[올해 최고의 영화! 세 번 넘게 봐도 질리지가 않네요.]는 99.44% 확률로 < 긍정적 > 입니다 ^^
[배경 음악이 영화의 분위기랑 너무 안 맞았습니다. 몰입에 방해가 됩니다.]는 80.10% 확률로 < 부정적 > 입니다 ㅠㅠ
[주연 배우가 신인인데 연기를 진짜 잘 하네요. 몰입감 ㅎㄷㄷ]는 93.90% 확률로 < 긍정적 > 입니다 ^^
[믿고 보는 감독이지만 이번에는 아니네요]는 73.86% 확률로 < 부정적 > 입니다 ㅠㅠ
[주연배우 때문에 봤어요]는 73.55% 확률로 < 부정적 > 입니다 ㅠㅠ
[오늘 뭐먹지?]는 94.76% 확률로 < 부정적 > 입니다 ㅠㅠ


In [14]:
predict_pos_neg("오늘 날씨가 왜이래")
predict_pos_neg("오늘 공원으로 가자")
predict_pos_neg("기분이 너무 좋아")
predict_pos_neg("안녕하세요")
predict_pos_neg("이건뭐")
predict_pos_neg("너무 우울해")
predict_pos_neg("반갑습니다")

[오늘 날씨가 왜이래]는 46.91% 확률로 < 중립 > 입니다.
[오늘 공원으로 가자]는 40.54% 확률로 < 중립 > 입니다.
[기분이 너무 좋아]는 90.61% 확률로 < 긍정적 > 입니다 ^^
[안녕하세요]는 47.13% 확률로 < 중립 > 입니다.
[이건뭐]는 94.80% 확률로 < 부정적 > 입니다 ㅠㅠ
[너무 우울해]는 77.50% 확률로 < 부정적 > 입니다 ㅠㅠ
[반갑습니다]는 47.13% 확률로 < 중립 > 입니다.
